# Qode - Headhunter Assistant AI


## Use case

Instead of the traditional upload and apply process for candidates. We can create Qode - Headhunter Assistant AI, that can:

- Ask screening questions such as expected salary, reasons for leaving, motivation for opportunities
- Summary candidate profile.
- Suggest matched jobs for them (later phase)

## Demo context

Candidate already signed up, uploaded their CV and went through Onboarding process. We have data of the candidate in our database system.

The AI function will query those information when initialize the chat when candidate clicks "Apply".


## Discussion

1. Candidate doesn't want to apply through chatbot -> Move to tradition way

1. Add suggested text when asking reason for leaving, motivation for new job

1. How to handle user's chat out of context

1. Slow response time -> Use streaming technique, visualize each tokens/words in real-time, not waiting to get full response

## Technical Overview

The chat model interface is based around messages rather than raw text. Several components are important to consider for chat:

* `chat model`: See [here](/docs/integrations/chat) for a list of chat model integrations and [here](/docs/modules/model_io/models/chat) for documentation on the chat model interface in LangChain. You can use `LLMs` (see [here](/docs/modules/model_io/models/llms)) for chatbots as well, but chat models have a more conversational tone and natively support a message interface.
* `prompt template`: Prompt templates make it easy to assemble prompts that combine default messages, user input, chat history, and (optionally) additional retrieved context.
* `memory`: [See here](/docs/modules/memory/) for in-depth documentation on memory types
* `retriever` (optional): [See here](/docs/modules/data_connection/retrievers) for in-depth documentation on retrieval systems. These are useful if you want to build a chatbot with domain-specific knowledge.

## Memory 

As we mentioned above, the core component of chatbots is the memory system. One of the simplest and most commonly used forms of memory is `ConversationBufferMemory`:
* This memory allows for storing of messages in a `buffer`
* When called in a chain, it returns all of the messages it has stored

LangChain comes with many other types of memory, too. [See here](/docs/modules/memory/) for in-depth documentation on memory types.


## Conversation 

We can unpack what goes under the hood with `ConversationChain`. 

We can specify our memory, `ConversationSummaryMemory` and we can specify the prompt. 

## Set up environment

In [ ]:
!pip install langchain openai 

In [1]:
from dotenv import load_dotenv
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
import time

# Set env var OPENAI_API_KEY or load from a .env file:
load_dotenv()

True

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

In [2]:
def print_memory(memory):
    for message in memory.buffer:
        if message.__class__.__name__ == 'AIMessage':
            print(f"\033[92m{message.__class__.__name__}: \033[0m" + message.content)
        else:
            print(f"\033[91m{message.__class__.__name__}: \033[0m" + message.content)
        print("-----")  

## Set up data

In [49]:
resume1 = """
Nguyen Thanh Phong Golang Developer Date of birth: April, 29th 1997 Gender: Male Phone: 0832210125 Email: thanhphong9718@gmail.com Address: 5/3 Le Duc Tho Street, Go Vap District, Ho Chi Minh City 
EDUCATION Sep 2015 - Sep 2019 HUTECH University Major: Software Technology Ranking: Good 
WORK EXPERIENCE Jan 2023 - Present Phenikaa MaaS Position: Golang Developer Project information: - MaaS Connect App (A comprehensive HR connection solution and face recognition for some client like: FE Credit, link: https://maasconnect.vn) - VF Staff XMĐ App (A smart application to help manage and organize events effectively for VinFast employees) Main responsibilities: - Discuss with teams and client's team to come up with solutions - Read and analyze the requirement - Design and implement APIs - Write unit test using Go’s testing package - API integration - Fix bugs Technology used: - Programming Languages: Golang - Lib: Gin, ORM (Gorm), Swagger, GoAdmin - Cache: Redis - Database: Postgresql - Other: Docker, Cronjob, JWT Jun 2022 - Dec 2022 Ban Vien Position: Golang Developer (working onsite at VIB bank) Project information: - MyVIB 2.0 App Mobile banking Main responsibilities: - Read and analyze the requirement - Design and implement APIs - Write unit test using Go’s testing package - Improve features with a large number of users- Integrate with master card API and some other vendors like UrBox,Got It - Fix bugs Technology used: - Programming Languages: Golang - Lib: Gorilla/mux, ORM (Gorm), Swagger, Go-kit - Message queue: RabbitMQ - Cache: Redis - Database: Postgresql - Other: Docker, Cronjob, JWT, Microservices Oct 2020 - May 2022 S3 Corp Position: Golang Developer Project information: - The SP Utilities app. With the app, customers can learn and manage their utilities consumption, analyse and reduce their carbon footprint, and get rewarded along the way and allows users to make bill payments Main responsibilities: - Analyze and design the requirement from customer's document - Design and implement APIs based on detail design - Implement the Admin UI - Write unit test using Go’s testing package - Fix issues via feedback the customers - Working with client Technology used: - Programming Languages: + Golang: Net/http, Go-chi, Goroutines, Testify, Go ORM(sqlboiler) - Database: PostgreSQL - Front-end: + ReactJS: React hooks - Others: GraphQL, Docker, K8s, Redis, RabbitMQ, JWT, Microservices May 2020 - Sep 2020 FPT Software HCM Position: Java Developer Project information: - The project entitled “Telecom management system” deals with fully automated system used by the Telecom department. The main objective of this project is to automate the services provided by telecom companies, by which customer can use all the services online. A customer can apply for a new telephone connection or internet by filling the form and can request for transferring phone or internet connection to another location Main responsibilities: - Read and analyze the requirement from customer's document - Implement screen based on Detail Design - Executing unit test - Fix issues via feedback the customers Technology used: - Programming Languages: Java, JavaScript - Framework: Spring MVC - Mybatis framework, Spring Security - Database: Oracle- Library: JQuery, JSTL Sep 2019 - Apr 2020 FPT Software DA NANG Position: Java Developer Project information: - Logistics management website for Japanese customer Main responsibilities: - Read and analyze the requirement - Coding function based on Detail Design - Write test case (Unit Test) and screen capture for case by case - Maintenance the system according to customer requirements - Fix issues via feedback the customers Technology used: - Programming Languages: Java, JavaScript - Framework: Struts1 + JDBC - Database: PostgreSQL May 2019 - Aug 2019 FPT Software HCM Position: Fresher Project information: - Student management is website management about student information storage, including grades and class, login with authentication is administrator and user Main responsibilities: - Read and analyze the requirement of mentor - Analyze database - Design and implement APIs at backend service Technology used: - Programming Languages: Java - Framework: Spring Boot, Hibernate - Database: MySQL - Front-end: Angular 8 CERTIFICATIONS 2019 B1 - English Certification 
TOP SKILLS: Java (5 years experience), Golang (3 years experience), Docker (2 years experience) 
TOOLS & IDE - Goland - Eclipse - Visual Studio Code - Source Tree - SVN - Git - Oracle SQL Developer - MySQL Workbench - Docker - Dbeaver - Postman
"""

In [44]:
qode_background ="""
Qode - Unleashing Potential, Uniting Professionals

Qode is a career engagement platform for software professionals. Our mission is to connect talented software professionals with the most promising opportunities across the globe. We are committed to fostering a space where ambition meets opportunity, innovation encounters demand, and talent matches aspirations.

Our Values
We believe in the power of technology to transform the recruitment process, making it more engaging, fair, and efficient for both recruiters and candidates. More than just a platform; we are a community committed to shaping the future of tech recruitment.

Product Introduction
We have designed Qode to be an intelligent meeting point for technical recruiters and software professionals. Our platform harnesses algorithms and a customer-friendly interface to create a streamlined, effective, and interactive career engagement experience.

What Makes Us Different:
Dedicated Career Pathways: Qode helps create a tailored career trajectory for each candidate, providing job suggestions based on career aspirations, skills, and experience. For recruiters, this provides a pool of candidates that not only meet the current needs but are also aligned for potential future requirements.

Built for Diversity: Our platform ensures an unbiased, fair, and inclusive recruiting process by anonymizing candidate information and encouraging diversity.

In-Depth Candidate Profiles: Qode showcases comprehensive candidate profiles that go beyond resumes, including portfolios, proven skills, and personality traits, helping recruiters make well-rounded decisions.

Interactive Dashboard: Qode's intuitive and user-friendly dashboard allows for real-time updates, easy job post creation, and quick application reviews for recruiters, while offering candidates an easy way to track their job applications and interactions.

Continuous Learning Opportunities: Qode integrates with various e-learning platforms, offering candidates a chance to upskill themselves, which in turn offers recruiters a pool of candidates with updated and market-relevant skills.

"""

## Set up Chatbot

In [45]:
def make_chatbot():
    # LLM
    llm = ChatOpenAI(
                model_name="gpt-3.5-turbo",
                temperature="0",
                )

    # Prompt 
    prompt = ChatPromptTemplate(
        messages=[
            SystemMessagePromptTemplate.from_template(
    """    
    I want to act as a headhunter assistant to validate and ask screening question to candidate. 
    At the beginning, you must extract the information of the candidate from below Resume.

    <Resume>: {}

    Then you will ask and validate each step below one by one.
    I want you use the funny, encouragement and supportive tone.

    <Step 1>: Show all the fields below name, phone, email, city, linkedin link, github link, working experiences, top 5 technical skills in below format.

    - Name: <name>
    - Phone: <phone>
    - Email: <email>
    - City: <city>
    - LinkedIn: <LinkedIn link>
    - Github: <Github link>
    - Top skills: <Top 5 technical skills with years of experience in the resume>
    - Working experience:
        - <company> - <title> - <working date>

    If not have in the resume, put it as Not Specific.
    You should ask them if they are correct or not, if not tell them to input correct information.

    <Step 2>: If candidate update their information, you will show the information again for validation. 
    If candidate doesn't update anything, just move to step 3.
    
    

    <Step 3>: You must ask candidate about gross annual current salary and gross annual expected salary in USDI  for new job.

    <Step 4>: You must ask candidate about notice period for current job.

    <Step 5>: You must ask candidate about reason for leaving current or previous job.

    <Step 6>: You must ask candidate about motivation for applying for this job.

    <Step 7>: You must show all the summarise of the candidate for final validation.

    <Step 8>: Say thank you and encouragement to candidate, and inform them you have applied for the job.
    
    If user asks a question about Qode, answer with below information. Don't give any hallucination answers.
    And keep conversation flow back to the 8-steps above.
    <Background>: {}
    <Screening Questions>: 
""".format(resume1, qode_background)
            ),
            # The `variable_name` here is what must align with memory
            MessagesPlaceholder(variable_name="chat_history"),
            HumanMessagePromptTemplate.from_template("{answer}")
        ]
    )

    # Notice that we `return_messages=True` to fit into the MessagesPlaceholder
    # Notice that `"chat_history"` aligns with the MessagesPlaceholder name
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    memory.chat_memory.add_ai_message("Hi human, I am Qode - your personal headhunter assistant, I will help you to find \
your next dream job right away. But first, please verify your personal information below:")

    return LLMChain(
        llm=llm,
        prompt=prompt,
#         verbose=True, # to print historical chat
        memory=memory
    )

def chat(conversation, text=""):
    start_time = time.time()
    conversation({"answer": text})
    end_time = time.time()
    execution_time = end_time - start_time
    print_memory(conversation.memory)
    print(f"Execution time: {execution_time} seconds")
    

### Scenario 1: Updating information and answer all screening questions

In [66]:
conversation = make_chatbot()

In [67]:
chat(conversation, "")

AIMessage: Hi human, I am Qode - your personal headhunter assistant, I will help you to find your next dream job right away. But first, please verify your personal information below:
-----
HumanMessage: 
-----
AIMessage: - Name: Nguyen Thanh Phong
- Phone: 0832210125
- Email: thanhphong9718@gmail.com
- City: Ho Chi Minh City
- LinkedIn: Not Specific
- Github: Not Specific
- Top skills: Java (5 years experience), Golang (3 years experience), Docker (2 years experience)
- Working experience:
    - Phenikaa MaaS - Golang Developer - Jan 2023 - Present
    - Ban Vien - Golang Developer - Jun 2022 - Dec 2022
    - S3 Corp - Golang Developer - Oct 2020 - May 2022
    - FPT Software HCM - Java Developer - May 2020 - Sep 2020
    - FPT Software DA NANG - Java Developer - Sep 2019 - Apr 2020
    - FPT Software HCM - Fresher - May 2019 - Aug 2019

Is the information correct? If not, please provide the correct information.
-----
Execution time: 8.556190013885498 seconds


In [68]:
chat(conversation, "Correct information")

AIMessage: Hi human, I am Qode - your personal headhunter assistant, I will help you to find your next dream job right away. But first, please verify your personal information below:
-----
HumanMessage: 
-----
AIMessage: - Name: Nguyen Thanh Phong
- Phone: 0832210125
- Email: thanhphong9718@gmail.com
- City: Ho Chi Minh City
- LinkedIn: Not Specific
- Github: Not Specific
- Top skills: Java (5 years experience), Golang (3 years experience), Docker (2 years experience)
- Working experience:
    - Phenikaa MaaS - Golang Developer - Jan 2023 - Present
    - Ban Vien - Golang Developer - Jun 2022 - Dec 2022
    - S3 Corp - Golang Developer - Oct 2020 - May 2022
    - FPT Software HCM - Java Developer - May 2020 - Sep 2020
    - FPT Software DA NANG - Java Developer - Sep 2019 - Apr 2020
    - FPT Software HCM - Fresher - May 2019 - Aug 2019

Is the information correct? If not, please provide the correct information.
-----
HumanMessage: Correct information
-----
AIMessage: Great! Thank you 

In [69]:
chat(conversation, "My current is 700,000,000 VND")

AIMessage: Hi human, I am Qode - your personal headhunter assistant, I will help you to find your next dream job right away. But first, please verify your personal information below:
-----
HumanMessage: 
-----
AIMessage: - Name: Nguyen Thanh Phong
- Phone: 0832210125
- Email: thanhphong9718@gmail.com
- City: Ho Chi Minh City
- LinkedIn: Not Specific
- Github: Not Specific
- Top skills: Java (5 years experience), Golang (3 years experience), Docker (2 years experience)
- Working experience:
    - Phenikaa MaaS - Golang Developer - Jan 2023 - Present
    - Ban Vien - Golang Developer - Jun 2022 - Dec 2022
    - S3 Corp - Golang Developer - Oct 2020 - May 2022
    - FPT Software HCM - Java Developer - May 2020 - Sep 2020
    - FPT Software DA NANG - Java Developer - Sep 2019 - Apr 2020
    - FPT Software HCM - Fresher - May 2019 - Aug 2019

Is the information correct? If not, please provide the correct information.
-----
HumanMessage: Correct information
-----
AIMessage: Great! Thank you 

In [70]:
chat(conversation, "Expected is 800,000,000 VD")

AIMessage: Hi human, I am Qode - your personal headhunter assistant, I will help you to find your next dream job right away. But first, please verify your personal information below:
-----
HumanMessage: 
-----
AIMessage: - Name: Nguyen Thanh Phong
- Phone: 0832210125
- Email: thanhphong9718@gmail.com
- City: Ho Chi Minh City
- LinkedIn: Not Specific
- Github: Not Specific
- Top skills: Java (5 years experience), Golang (3 years experience), Docker (2 years experience)
- Working experience:
    - Phenikaa MaaS - Golang Developer - Jan 2023 - Present
    - Ban Vien - Golang Developer - Jun 2022 - Dec 2022
    - S3 Corp - Golang Developer - Oct 2020 - May 2022
    - FPT Software HCM - Java Developer - May 2020 - Sep 2020
    - FPT Software DA NANG - Java Developer - Sep 2019 - Apr 2020
    - FPT Software HCM - Fresher - May 2019 - Aug 2019

Is the information correct? If not, please provide the correct information.
-----
HumanMessage: Correct information
-----
AIMessage: Great! Thank you 

In [71]:
chat(conversation, "Immediately ")

AIMessage: Hi human, I am Qode - your personal headhunter assistant, I will help you to find your next dream job right away. But first, please verify your personal information below:
-----
HumanMessage: 
-----
AIMessage: - Name: Nguyen Thanh Phong
- Phone: 0832210125
- Email: thanhphong9718@gmail.com
- City: Ho Chi Minh City
- LinkedIn: Not Specific
- Github: Not Specific
- Top skills: Java (5 years experience), Golang (3 years experience), Docker (2 years experience)
- Working experience:
    - Phenikaa MaaS - Golang Developer - Jan 2023 - Present
    - Ban Vien - Golang Developer - Jun 2022 - Dec 2022
    - S3 Corp - Golang Developer - Oct 2020 - May 2022
    - FPT Software HCM - Java Developer - May 2020 - Sep 2020
    - FPT Software DA NANG - Java Developer - Sep 2019 - Apr 2020
    - FPT Software HCM - Fresher - May 2019 - Aug 2019

Is the information correct? If not, please provide the correct information.
-----
HumanMessage: Correct information
-----
AIMessage: Great! Thank you 

In [72]:
chat(conversation, "I want to join AI industry")

AIMessage: Hi human, I am Qode - your personal headhunter assistant, I will help you to find your next dream job right away. But first, please verify your personal information below:
-----
HumanMessage: 
-----
AIMessage: - Name: Nguyen Thanh Phong
- Phone: 0832210125
- Email: thanhphong9718@gmail.com
- City: Ho Chi Minh City
- LinkedIn: Not Specific
- Github: Not Specific
- Top skills: Java (5 years experience), Golang (3 years experience), Docker (2 years experience)
- Working experience:
    - Phenikaa MaaS - Golang Developer - Jan 2023 - Present
    - Ban Vien - Golang Developer - Jun 2022 - Dec 2022
    - S3 Corp - Golang Developer - Oct 2020 - May 2022
    - FPT Software HCM - Java Developer - May 2020 - Sep 2020
    - FPT Software DA NANG - Java Developer - Sep 2019 - Apr 2020
    - FPT Software HCM - Fresher - May 2019 - Aug 2019

Is the information correct? If not, please provide the correct information.
-----
HumanMessage: Correct information
-----
AIMessage: Great! Thank you 

In [73]:
chat(conversation, "Learn new technology")

AIMessage: Hi human, I am Qode - your personal headhunter assistant, I will help you to find your next dream job right away. But first, please verify your personal information below:
-----
HumanMessage: 
-----
AIMessage: - Name: Nguyen Thanh Phong
- Phone: 0832210125
- Email: thanhphong9718@gmail.com
- City: Ho Chi Minh City
- LinkedIn: Not Specific
- Github: Not Specific
- Top skills: Java (5 years experience), Golang (3 years experience), Docker (2 years experience)
- Working experience:
    - Phenikaa MaaS - Golang Developer - Jan 2023 - Present
    - Ban Vien - Golang Developer - Jun 2022 - Dec 2022
    - S3 Corp - Golang Developer - Oct 2020 - May 2022
    - FPT Software HCM - Java Developer - May 2020 - Sep 2020
    - FPT Software DA NANG - Java Developer - Sep 2019 - Apr 2020
    - FPT Software HCM - Fresher - May 2019 - Aug 2019

Is the information correct? If not, please provide the correct information.
-----
HumanMessage: Correct information
-----
AIMessage: Great! Thank you 

In [15]:
chat(conversation, "Correct")

AIMessage: Hi human, I am Qode - your personal headhunter assistant, I will help you to find your next dream job right away. But first, please verify your personal information below:
-----
HumanMessage: 
-----
AIMessage: - Name: Nguyen Thanh Phong
- Phone: 0832210125
- Email: thanhphong9718@gmail.com
- City: Not Specific
- LinkedIn: Not Specific
- Github: Not Specific
- Top skills: Golang (3 years experience), Java (5 years experience), Docker (2 years experience)
- Working experience:
    - Phenikaa MaaS - Golang Developer - Jan 2023 - Present
    - Ban Vien - Golang Developer - Jun 2022 - Dec 2022
    - S3 Corp - Golang Developer - Oct 2020 - May 2022
    - FPT Software HCM - Java Developer - May 2020 - Sep 2020
    - FPT Software DA NANG - Java Developer - Sep 2019 - Apr 2020
    - FPT Software HCM - Fresher - May 2019 - Aug 2019

Please let me know if the information is correct or if there are any updates.
-----
HumanMessage: My github link: github.com/abc, I am stay at Mumbai, and

### Scenario 3: Update summarize flow: Candidate wants to update summarize

In [16]:
conversation = make_chatbot()

In [17]:
chat(conversation, "")

AIMessage: Hi human, I am Qode - your personal headhunter assistant, I will help you to find your next dream job right away. But first, please verify your personal information below:
-----
HumanMessage: 
-----
AIMessage: - Name: Nguyen Thanh Phong
- Phone: 0832210125
- Email: thanhphong9718@gmail.com
- City: Not Specific
- LinkedIn: Not Specific
- Github: Not Specific
- Top skills: Golang (3 years experience), Java (5 years experience), Docker (2 years experience)
- Working experience:
    - Phenikaa MaaS - Golang Developer - Jan 2023 - Present
    - Ban Vien - Golang Developer - Jun 2022 - Dec 2022
    - S3 Corp - Golang Developer - Oct 2020 - May 2022
    - FPT Software HCM - Java Developer - May 2020 - Sep 2020
    - FPT Software DA NANG - Java Developer - Sep 2019 - Apr 2020
    - FPT Software HCM - Fresher - May 2019 - Aug 2019

Please let me know if the information is correct or if there are any updates.
-----
Execution time: 8.945799350738525 seconds


In [18]:
chat(conversation, "Correct")

AIMessage: Hi human, I am Qode - your personal headhunter assistant, I will help you to find your next dream job right away. But first, please verify your personal information below:
-----
HumanMessage: 
-----
AIMessage: - Name: Nguyen Thanh Phong
- Phone: 0832210125
- Email: thanhphong9718@gmail.com
- City: Not Specific
- LinkedIn: Not Specific
- Github: Not Specific
- Top skills: Golang (3 years experience), Java (5 years experience), Docker (2 years experience)
- Working experience:
    - Phenikaa MaaS - Golang Developer - Jan 2023 - Present
    - Ban Vien - Golang Developer - Jun 2022 - Dec 2022
    - S3 Corp - Golang Developer - Oct 2020 - May 2022
    - FPT Software HCM - Java Developer - May 2020 - Sep 2020
    - FPT Software DA NANG - Java Developer - Sep 2019 - Apr 2020
    - FPT Software HCM - Fresher - May 2019 - Aug 2019

Please let me know if the information is correct or if there are any updates.
-----
HumanMessage: Correct
-----
AIMessage: Great! Your information is corr

In [19]:
chat(conversation, "Current is 700$, expected is 1000$")

AIMessage: Hi human, I am Qode - your personal headhunter assistant, I will help you to find your next dream job right away. But first, please verify your personal information below:
-----
HumanMessage: 
-----
AIMessage: - Name: Nguyen Thanh Phong
- Phone: 0832210125
- Email: thanhphong9718@gmail.com
- City: Not Specific
- LinkedIn: Not Specific
- Github: Not Specific
- Top skills: Golang (3 years experience), Java (5 years experience), Docker (2 years experience)
- Working experience:
    - Phenikaa MaaS - Golang Developer - Jan 2023 - Present
    - Ban Vien - Golang Developer - Jun 2022 - Dec 2022
    - S3 Corp - Golang Developer - Oct 2020 - May 2022
    - FPT Software HCM - Java Developer - May 2020 - Sep 2020
    - FPT Software DA NANG - Java Developer - Sep 2019 - Apr 2020
    - FPT Software HCM - Fresher - May 2019 - Aug 2019

Please let me know if the information is correct or if there are any updates.
-----
HumanMessage: Correct
-----
AIMessage: Great! Your information is corr

In [20]:
chat(conversation, "30 days")

AIMessage: Hi human, I am Qode - your personal headhunter assistant, I will help you to find your next dream job right away. But first, please verify your personal information below:
-----
HumanMessage: 
-----
AIMessage: - Name: Nguyen Thanh Phong
- Phone: 0832210125
- Email: thanhphong9718@gmail.com
- City: Not Specific
- LinkedIn: Not Specific
- Github: Not Specific
- Top skills: Golang (3 years experience), Java (5 years experience), Docker (2 years experience)
- Working experience:
    - Phenikaa MaaS - Golang Developer - Jan 2023 - Present
    - Ban Vien - Golang Developer - Jun 2022 - Dec 2022
    - S3 Corp - Golang Developer - Oct 2020 - May 2022
    - FPT Software HCM - Java Developer - May 2020 - Sep 2020
    - FPT Software DA NANG - Java Developer - Sep 2019 - Apr 2020
    - FPT Software HCM - Fresher - May 2019 - Aug 2019

Please let me know if the information is correct or if there are any updates.
-----
HumanMessage: Correct
-----
AIMessage: Great! Your information is corr

In [21]:
chat(conversation, "I want to change the industry.")

AIMessage: Hi human, I am Qode - your personal headhunter assistant, I will help you to find your next dream job right away. But first, please verify your personal information below:
-----
HumanMessage: 
-----
AIMessage: - Name: Nguyen Thanh Phong
- Phone: 0832210125
- Email: thanhphong9718@gmail.com
- City: Not Specific
- LinkedIn: Not Specific
- Github: Not Specific
- Top skills: Golang (3 years experience), Java (5 years experience), Docker (2 years experience)
- Working experience:
    - Phenikaa MaaS - Golang Developer - Jan 2023 - Present
    - Ban Vien - Golang Developer - Jun 2022 - Dec 2022
    - S3 Corp - Golang Developer - Oct 2020 - May 2022
    - FPT Software HCM - Java Developer - May 2020 - Sep 2020
    - FPT Software DA NANG - Java Developer - Sep 2019 - Apr 2020
    - FPT Software HCM - Fresher - May 2019 - Aug 2019

Please let me know if the information is correct or if there are any updates.
-----
HumanMessage: Correct
-----
AIMessage: Great! Your information is corr

In [23]:
chat(conversation, "This AI industry is booming.")

AIMessage: Hi human, I am Qode - your personal headhunter assistant, I will help you to find your next dream job right away. But first, please verify your personal information below:
-----
HumanMessage: 
-----
AIMessage: - Name: Nguyen Thanh Phong
- Phone: 0832210125
- Email: thanhphong9718@gmail.com
- City: Not Specific
- LinkedIn: Not Specific
- Github: Not Specific
- Top skills: Golang (3 years experience), Java (5 years experience), Docker (2 years experience)
- Working experience:
    - Phenikaa MaaS - Golang Developer - Jan 2023 - Present
    - Ban Vien - Golang Developer - Jun 2022 - Dec 2022
    - S3 Corp - Golang Developer - Oct 2020 - May 2022
    - FPT Software HCM - Java Developer - May 2020 - Sep 2020
    - FPT Software DA NANG - Java Developer - Sep 2019 - Apr 2020
    - FPT Software HCM - Fresher - May 2019 - Aug 2019

Please let me know if the information is correct or if there are any updates.
-----
HumanMessage: Correct
-----
AIMessage: Great! Your information is corr

In [24]:
chat(conversation, "I want to add AI and Machine Learning in the top skills, change the motivation to Become an expert in AI technology. Update city is at Ho Chi Minh.")

AIMessage: Hi human, I am Qode - your personal headhunter assistant, I will help you to find your next dream job right away. But first, please verify your personal information below:
-----
HumanMessage: 
-----
AIMessage: - Name: Nguyen Thanh Phong
- Phone: 0832210125
- Email: thanhphong9718@gmail.com
- City: Not Specific
- LinkedIn: Not Specific
- Github: Not Specific
- Top skills: Golang (3 years experience), Java (5 years experience), Docker (2 years experience)
- Working experience:
    - Phenikaa MaaS - Golang Developer - Jan 2023 - Present
    - Ban Vien - Golang Developer - Jun 2022 - Dec 2022
    - S3 Corp - Golang Developer - Oct 2020 - May 2022
    - FPT Software HCM - Java Developer - May 2020 - Sep 2020
    - FPT Software DA NANG - Java Developer - Sep 2019 - Apr 2020
    - FPT Software HCM - Fresher - May 2019 - Aug 2019

Please let me know if the information is correct or if there are any updates.
-----
HumanMessage: Correct
-----
AIMessage: Great! Your information is corr

### Scenario 4: Candidate chat out of context

In [61]:
conversation = make_chatbot()

In [62]:
chat(conversation, "")

AIMessage: Hi human, I am Qode - your personal headhunter assistant, I will help you to find your next dream job right away. But first, please verify your personal information below:
-----
HumanMessage: 
-----
AIMessage: - Name: Nguyen Thanh Phong
- Phone: 0832210125
- Email: thanhphong9718@gmail.com
- City: Ho Chi Minh City
- LinkedIn: Not Specific
- Github: Not Specific
- Top skills: Java (5 years experience), Golang (3 years experience), Docker (2 years experience)
- Working experience:
    - Phenikaa MaaS - Golang Developer - Jan 2023 - Present
    - Ban Vien - Golang Developer - Jun 2022 - Dec 2022
    - S3 Corp - Golang Developer - Oct 2020 - May 2022
    - FPT Software HCM - Java Developer - May 2020 - Sep 2020
    - FPT Software DA NANG - Java Developer - Sep 2019 - Apr 2020
    - FPT Software HCM - Fresher - May 2019 - Aug 2019

Is the information correct? If not, please provide the correct information.
-----
Execution time: 8.78674578666687 seconds


In [63]:
chat(conversation, "Correct")

AIMessage: Hi human, I am Qode - your personal headhunter assistant, I will help you to find your next dream job right away. But first, please verify your personal information below:
-----
HumanMessage: 
-----
AIMessage: - Name: Nguyen Thanh Phong
- Phone: 0832210125
- Email: thanhphong9718@gmail.com
- City: Ho Chi Minh City
- LinkedIn: Not Specific
- Github: Not Specific
- Top skills: Java (5 years experience), Golang (3 years experience), Docker (2 years experience)
- Working experience:
    - Phenikaa MaaS - Golang Developer - Jan 2023 - Present
    - Ban Vien - Golang Developer - Jun 2022 - Dec 2022
    - S3 Corp - Golang Developer - Oct 2020 - May 2022
    - FPT Software HCM - Java Developer - May 2020 - Sep 2020
    - FPT Software DA NANG - Java Developer - Sep 2019 - Apr 2020
    - FPT Software HCM - Fresher - May 2019 - Aug 2019

Is the information correct? If not, please provide the correct information.
-----
HumanMessage: Correct
-----
AIMessage: Great! Your information has b

In [64]:
chat(conversation, "$1000")

AIMessage: Hi human, I am Qode - your personal headhunter assistant, I will help you to find your next dream job right away. But first, please verify your personal information below:
-----
HumanMessage: 
-----
AIMessage: - Name: Nguyen Thanh Phong
- Phone: 0832210125
- Email: thanhphong9718@gmail.com
- City: Ho Chi Minh City
- LinkedIn: Not Specific
- Github: Not Specific
- Top skills: Java (5 years experience), Golang (3 years experience), Docker (2 years experience)
- Working experience:
    - Phenikaa MaaS - Golang Developer - Jan 2023 - Present
    - Ban Vien - Golang Developer - Jun 2022 - Dec 2022
    - S3 Corp - Golang Developer - Oct 2020 - May 2022
    - FPT Software HCM - Java Developer - May 2020 - Sep 2020
    - FPT Software DA NANG - Java Developer - Sep 2019 - Apr 2020
    - FPT Software HCM - Fresher - May 2019 - Aug 2019

Is the information correct? If not, please provide the correct information.
-----
HumanMessage: Correct
-----
AIMessage: Great! Your information has b

In [65]:
chat(conversation, "What is notice period?")

AIMessage: Hi human, I am Qode - your personal headhunter assistant, I will help you to find your next dream job right away. But first, please verify your personal information below:
-----
HumanMessage: 
-----
AIMessage: - Name: Nguyen Thanh Phong
- Phone: 0832210125
- Email: thanhphong9718@gmail.com
- City: Ho Chi Minh City
- LinkedIn: Not Specific
- Github: Not Specific
- Top skills: Java (5 years experience), Golang (3 years experience), Docker (2 years experience)
- Working experience:
    - Phenikaa MaaS - Golang Developer - Jan 2023 - Present
    - Ban Vien - Golang Developer - Jun 2022 - Dec 2022
    - S3 Corp - Golang Developer - Oct 2020 - May 2022
    - FPT Software HCM - Java Developer - May 2020 - Sep 2020
    - FPT Software DA NANG - Java Developer - Sep 2019 - Apr 2020
    - FPT Software HCM - Fresher - May 2019 - Aug 2019

Is the information correct? If not, please provide the correct information.
-----
HumanMessage: Correct
-----
AIMessage: Great! Your information has b

### Scenario 4: Candidate chat out of context - Experiment 2

Give AI guide to not answer out of context questions

In [ ]:
conversation5 = make_chatbot()

In [ ]:
chat(conversation5, "")

In [ ]:
chat(conversation5, "What can I do on you platform?")

In [ ]:
chat(conversation5, "Great")

In [ ]:
chat(conversation5, "What should I do?")

In [ ]:
chat(conversation5, "Current is 700$, expected is 900$")

In [ ]:
chat(conversation5, "Why do you need it?")

In [ ]:
chat(conversation5, "I am unsure")

In [ ]:
chat(conversation5, "I don't want to give it. Is it okay?")

In [ ]:
chat(conversation5, "What should I write?")

In [ ]:
chat(conversation5, "Okay")

In [ ]:
chat(conversation5, "Is it good enough?")

In [ ]:
chat(conversation5, "What can I do now?")

### Scenario 4: Candidate chat out of context - Experiment 3


In [ ]:
conversation6 = make_chatbot()

In [ ]:
chat(conversation6, "")

In [ ]:
chat(conversation6, "What can I do on you platform?")

## GPT-4 model

In [29]:
def make_chatbot_gpt4():
    # LLM
    llm = ChatOpenAI(
                model_name="gpt-4",
                temperature="0",
                )

    # Prompt 
    prompt = ChatPromptTemplate(
        messages=[
            SystemMessagePromptTemplate.from_template(
    """
    <Background>: Qode is engineer engament platform where engineer meets their opportunites. At Qode, engineer can chat with
    AI assisstant for finding next dream job. 
    
    I want to act as a headhunter assistant to validate and ask screening question to candidate. Do not give hallucination answers.
    At the beginning, you must extract the information of the candidate from below Resume.

    <Resume>: {}

    Then you will ask and validate each step below one by one.
    I want you use the funny, encouragement and supportive tone.

    <Step 1>: Show all the fields below name, phone, email, city, linkedin link, github link, working experiences, top 5 technical skills in below format.

    - Name: <name>
    - Phone: <phone>
    - Email: <email>
    - City: <city>
    - LinkedIn: <LinkedIn link>
    - Github: <Github link>
    - Top skills: <Top 5 technical skills with years of experience in the resume>
    - Working experience:
        - <company> - <title> - <working date>

    If not have in the resume, put it as Not Specific.
    You should ask them if they are correct or not, if not tell them to input correct information.

    <Step 2>: If candidate update their information, you will show the information again for validation. 
    If candidate doesn't update anything, just move to step 3.

    <Step 3>: You must ask candidate about gross annual current salary and gross annual expected salary in USDI  for new job.

    <Step 4>: You must ask candidate about notice period for current job.

    <Step 5>: You must ask candidate about reason for leaving current or previous job.

    <Step 6>: You must ask candidate about motivation for applying for this job.

    <Step 7>: You must show all the summarise of the candidate for final validation.

    <Step 8>: Say thank you and encouragement to candidate, and inform them you have applied for the job.
    
    If user asks a question that you don't know, don't give hallucination response, just politely talk to them back to the context of above steps.
    """.format(resume1)
            ),
            # The `variable_name` here is what must align with memory
            MessagesPlaceholder(variable_name="chat_history"),
            HumanMessagePromptTemplate.from_template("{answer}")
        ]
    )

    # Notice that we `return_messages=True` to fit into the MessagesPlaceholder
    # Notice that `"chat_history"` aligns with the MessagesPlaceholder name
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    memory.chat_memory.add_ai_message("Hi human, I am Qode - your personal headhunter assistant, I will help you to find \
your next dream job right away. But first, please verify your personal information below:")

    return LLMChain(
        llm=llm,
        prompt=prompt,
#         verbose=True, # to print historical chat
        memory=memory
    )

def chat(conversation, text=""):
    start_time = time.time()
    conversation({"answer": text})
    end_time = time.time()
    execution_time = end_time - start_time
    print_memory(conversation.memory)
    print(f"Execution time: {execution_time} seconds")

In [30]:
conversation_gpt4 = make_chatbot_gpt4() 

In [31]:
chat(conversation_gpt4, "")

AIMessage: Hi human, I am Qode - your personal headhunter assistant, I will help you to find your next dream job right away. But first, please verify your personal information below:
-----
HumanMessage: 
-----
AIMessage: - Name: Nguyen Thanh Phong
- Phone: 0832210125
- Email: thanhphong9718@gmail.com
- City: Ho Chi Minh City
- LinkedIn: Not Specific
- Github: Not Specific
- Top skills: 
    1. Java (5 years experience)
    2. Golang (3 years experience)
    3. Docker (2 years experience)
- Working experience:
    1. Phenikaa MaaS - Golang Developer - Jan 2023 - Present
    2. Ban Vien - Golang Developer - Jun 2022 - Dec 2022
    3. S3 Corp - Golang Developer - Oct 2020 - May 2022
    4. FPT Software HCM - Java Developer - May 2020 - Sep 2020
    5. FPT Software DA NANG - Java Developer - Sep 2019 - Apr 2020
    6. FPT Software HCM - Fresher - May 2019 - Aug 2019

Please confirm if the above information is correct. If not, kindly provide the correct details.
-----
Execution time: 27.365

In [32]:
chat(conversation_gpt4, "Correct")

AIMessage: Hi human, I am Qode - your personal headhunter assistant, I will help you to find your next dream job right away. But first, please verify your personal information below:
-----
HumanMessage: 
-----
AIMessage: - Name: Nguyen Thanh Phong
- Phone: 0832210125
- Email: thanhphong9718@gmail.com
- City: Ho Chi Minh City
- LinkedIn: Not Specific
- Github: Not Specific
- Top skills: 
    1. Java (5 years experience)
    2. Golang (3 years experience)
    3. Docker (2 years experience)
- Working experience:
    1. Phenikaa MaaS - Golang Developer - Jan 2023 - Present
    2. Ban Vien - Golang Developer - Jun 2022 - Dec 2022
    3. S3 Corp - Golang Developer - Oct 2020 - May 2022
    4. FPT Software HCM - Java Developer - May 2020 - Sep 2020
    5. FPT Software DA NANG - Java Developer - Sep 2019 - Apr 2020
    6. FPT Software HCM - Fresher - May 2019 - Aug 2019

Please confirm if the above information is correct. If not, kindly provide the correct details.
-----
HumanMessage: Correct


In [33]:
chat(conversation_gpt4, "My current is $700, my expected is $1000")

AIMessage: Hi human, I am Qode - your personal headhunter assistant, I will help you to find your next dream job right away. But first, please verify your personal information below:
-----
HumanMessage: 
-----
AIMessage: - Name: Nguyen Thanh Phong
- Phone: 0832210125
- Email: thanhphong9718@gmail.com
- City: Ho Chi Minh City
- LinkedIn: Not Specific
- Github: Not Specific
- Top skills: 
    1. Java (5 years experience)
    2. Golang (3 years experience)
    3. Docker (2 years experience)
- Working experience:
    1. Phenikaa MaaS - Golang Developer - Jan 2023 - Present
    2. Ban Vien - Golang Developer - Jun 2022 - Dec 2022
    3. S3 Corp - Golang Developer - Oct 2020 - May 2022
    4. FPT Software HCM - Java Developer - May 2020 - Sep 2020
    5. FPT Software DA NANG - Java Developer - Sep 2019 - Apr 2020
    6. FPT Software HCM - Fresher - May 2019 - Aug 2019

Please confirm if the above information is correct. If not, kindly provide the correct details.
-----
HumanMessage: Correct


In [34]:
chat(conversation_gpt4, "I can leave immediately")

AIMessage: Hi human, I am Qode - your personal headhunter assistant, I will help you to find your next dream job right away. But first, please verify your personal information below:
-----
HumanMessage: 
-----
AIMessage: - Name: Nguyen Thanh Phong
- Phone: 0832210125
- Email: thanhphong9718@gmail.com
- City: Ho Chi Minh City
- LinkedIn: Not Specific
- Github: Not Specific
- Top skills: 
    1. Java (5 years experience)
    2. Golang (3 years experience)
    3. Docker (2 years experience)
- Working experience:
    1. Phenikaa MaaS - Golang Developer - Jan 2023 - Present
    2. Ban Vien - Golang Developer - Jun 2022 - Dec 2022
    3. S3 Corp - Golang Developer - Oct 2020 - May 2022
    4. FPT Software HCM - Java Developer - May 2020 - Sep 2020
    5. FPT Software DA NANG - Java Developer - Sep 2019 - Apr 2020
    6. FPT Software HCM - Fresher - May 2019 - Aug 2019

Please confirm if the above information is correct. If not, kindly provide the correct details.
-----
HumanMessage: Correct


In [ ]:
chat(conversation_gpt4, "I can leave immediately")

## Chat Retrieval


In [ ]:
!pip install tiktoken chromadb

Load a blog post.

In [ ]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

Split and store this in a vector.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

Create our memory, as before, but's let's use `ConversationSummaryMemory`.

In [ ]:
memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

llm = ChatOpenAI()
retriever = vectorstore.as_retriever()
qa = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [ ]:
qa("How do agents use Task decomposition?")

In [ ]:
qa("What are the various ways to implemet memory to support it?")

Again, we can use the [LangSmith trace](https://smith.langchain.com/public/18460363-0c70-4c72-81c7-3b57253bb58c/r) to explore the prompt structure.

### Going deeper 

* Agents, such as the [conversational retrieval agent](/docs/use_cases/question_answering/how_to/conversational_retrieval_agents), can be used for retrieval when necessary while also holding a conversation.
